---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
#max_degrees = [max(G.degree().values()) for G in P1_Graphs]
#max_degrees

In [4]:
#cluster_coefficients = [nx.average_clustering(G) for G in P1_Graphs]
#cluster_coefficients

In [5]:
#avg_path_lens = [nx.average_shortest_path_length(G) for G in P1_Graphs]
#avg_path_lens

In [6]:
def graph_identification(Gs = P1_Graphs):
    
    graph_types = []
    for G in Gs:
        max_degree = max(G.degree().values())
        cluster_coeff = nx.average_clustering(G)
        avg_path_len = nx.average_shortest_path_length(G)
        if max_degree > 15:
            graph_types.append('PA')
        elif cluster_coeff > 0.35:
            graph_types.append('SW_L')
        elif avg_path_len < 5.5:
            graph_types.append('SW_H')
        else:
            graph_types.append('???')
    
    return graph_types

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
## Node information into a dataframe
#df = pd.DataFrame(index = G.nodes())
#
### Dependent variable
#df['ManagementSalary'] = pd.Series(nx.get_node_attributes(
#                                      G, 'ManagementSalary'))
#train_df = df[~np.isnan(df['ManagementSalary'])] ## 753
#
### Get the deparment feature variable, one-hot encoded since it's categorical
#df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
#enc = OneHotEncoder(handle_unknown='ignore')
#enc = enc.fit(df[['Department']])
#trans = enc.transform(df[['Department']]).toarray()
#trans_df = pd.DataFrame(trans, 
#                columns = [f'Department_{x}' for x in range(42)],
#                index = df.index)
### only keep departments that have more than 10  
#dept_cnts = (df.groupby('Department').agg({'ManagementSalary': ['mean', 'count']}))
#dept_cnts = dept_cnts[dept_cnts[('ManagementSalary', 'mean')] > 10].copy()
#trans_df = trans_df.drop([f'Department_{x}' 
#                         for x in dept_cnts.index], axis = 1)
#feature_cols = list(trans_df.columns)
#df = pd.concat([df, trans_df], axis = 1)
#
#### NetworkX stats on nodes
#df['clustering'] = pd.Series(nx.clustering(G))
#df['degree'] = pd.Series(G.degree())
#df['betweenness'] = pd.Series(
#                     nx.betweenness_centrality(G, 
#                       normalized = True, endpoints = False, k = None))
#feature_cols.extend(['clustering', 'degree', 'betweenness'])
#
### Split into training and missing management salary (for predictions)
#train_df = df[~np.isnan(df['ManagementSalary'])].copy() ## 753
#pred_df = df[np.isnan(df['ManagementSalary'])].copy() ## 252
#
#X = train_df[feature_cols]
#y = train_df['ManagementSalary']
#
#log_model = LogisticRegression(random_state=0).fit(X, y)
#train_df['fitted_probs'] = [pred[1] for pred in log_model.predict_proba(X)]
#
### make predictions
#X_pred = pred_df[feature_cols]
#preds = pd.Series([pred[1] for pred in log_model.predict_proba(X_pred)])

In [9]:
def salary_predictions():
    
    ## Node information into a dataframe
    df = pd.DataFrame(index = G.nodes())

    ## Dependent variable
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(
                                          G, 'ManagementSalary'))
    train_df = df[~np.isnan(df['ManagementSalary'])] ## 753

    ## Get the deparment feature variable, one-hot encoded since it's categorical
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    enc = OneHotEncoder(handle_unknown='ignore')
    enc = enc.fit(df[['Department']])
    trans = enc.transform(df[['Department']]).toarray()
    trans_df = pd.DataFrame(trans, 
                    columns = [f'Department_{x}' for x in range(42)],
                    index = df.index)
    ## only keep departments that have more than 10  
    dept_cnts = (df.groupby('Department').agg({'ManagementSalary': ['mean', 'count']}))
    dept_cnts = dept_cnts[dept_cnts[('ManagementSalary', 'mean')] > 10].copy()
    trans_df = trans_df.drop([f'Department_{x}' 
                             for x in dept_cnts.index], axis = 1)
    feature_cols = list(trans_df.columns)
    df = pd.concat([df, trans_df], axis = 1)

    ### NetworkX stats on nodes
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['betweenness'] = pd.Series(
                         nx.betweenness_centrality(G, 
                           normalized = True, endpoints = False, k = None))
    feature_cols.extend(['clustering', 'degree', 'betweenness'])

    ## Split into training and missing management salary (for predictions)
    train_df = df[~np.isnan(df['ManagementSalary'])].copy() ## 753
    pred_df = df[np.isnan(df['ManagementSalary'])].copy() ## 252

    X = train_df[feature_cols]
    y = train_df['ManagementSalary']

    log_model = LogisticRegression(random_state=0).fit(X, y)
    train_df['fitted_probs'] = [pred[1] for pred in log_model.predict_proba(X)]

    ## make predictions
    X_pred = pred_df[feature_cols]
    preds = pd.Series([pred[1] for pred in log_model.predict_proba(X_pred)])
    
    return preds

salary_predictions()

0      0.064767
1      0.730604
2      0.985019
3      0.066469
4      0.379316
5      0.079664
6      0.337599
7      0.398688
8      0.236709
9      0.159043
10     0.153998
11     0.306863
12     0.125737
13     0.358402
14     0.288767
15     0.226134
16     0.995901
17     0.479884
18     0.064397
19     0.117568
20     0.036925
21     0.031281
22     0.204670
23     0.090819
24     0.783538
25     0.039226
26     0.282302
27     0.916800
28     0.046600
29     0.359391
         ...   
222    0.032614
223    0.086437
224    0.020567
225    0.062157
226    0.042473
227    0.057835
228    0.049341
229    0.027924
230    0.059063
231    0.024457
232    0.040760
233    0.090200
234    0.142686
235    0.027519
236    0.059765
237    0.014863
238    0.014239
239    0.113039
240    0.087660
241    0.081028
242    0.032825
243    0.026782
244    0.134450
245    0.047242
246    0.089584
247    0.016175
248    0.030319
249    0.007956
250    0.042129
251    0.132956
Length: 252, dtype: floa

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
#### DFs with measures of interest used to predict
#jaccard_df = pd.DataFrame(list(nx.jaccard_coefficient(G)),
#                         columns = ['source', 'dest', 'jaccard'])
#jaccard_df.index = [(s, d) for s, d in zip(jaccard_df['source'],
#                                           jaccard_df['dest'])]
#jaccard_df = jaccard_df.drop(['source', 'dest'], axis = 1)
#
#
#resource_df = pd.DataFrame(list(nx.adamic_adar_index(G)),
#                         columns = ['source', 'dest', 'resource'])
#resource_df.index = [(s, d) for s, d in zip(resource_df['source'],
#                                           resource_df['dest'])]
#resource_df = resource_df.drop(['source', 'dest'], axis = 1)
#
#
#preferential_df = pd.DataFrame(list(nx.preferential_attachment(G)),
#                         columns = ['source', 'dest', 'preferential'])
#preferential_df.index = [(s, d) for s, d in zip(preferential_df['source'],
#                                           preferential_df['dest'])]
#preferential_df = preferential_df.drop(['source', 'dest'], axis = 1)
#
#feature_cols = ['jaccard', 'resource', 'preferential']
#feature_df = pd.concat([jaccard_df,
#                        resource_df,
#                        preferential_df], axis = 1)
#
#### Combine the dfs
#df = future_connections.join(feature_df, how = 'inner')
#
### Split into training and prediction
#train_df = df[~np.isnan(df['Future Connection'])]
#pred_df = df[np.isnan(df['Future Connection'])]
#
#X = train_df[feature_cols]
#y = train_df['Future Connection']
#
#log_model = LogisticRegression(random_state=0).fit(X, y)
#train_df['fitted_probs'] = [pred[1] for pred in log_model.predict_proba(X)]
#
### make predictions
#X_pred = pred_df[feature_cols]
#X_pred = X_pred.fillna(0)
#preds = pd.Series([pred[1] for pred in log_model.predict_proba(X_pred)],
#                 index = X_pred.index)
#
#preds

In [12]:
def new_connections_predictions():
    
    #### DFs with measures of interest used to predict
    jaccard_df = pd.DataFrame(list(nx.jaccard_coefficient(G)),
                             columns = ['source', 'dest', 'jaccard'])
    jaccard_df.index = [(s, d) for s, d in zip(jaccard_df['source'],
                                               jaccard_df['dest'])]
    jaccard_df = jaccard_df.drop(['source', 'dest'], axis = 1)


    resource_df = pd.DataFrame(list(nx.adamic_adar_index(G)),
                             columns = ['source', 'dest', 'resource'])
    resource_df.index = [(s, d) for s, d in zip(resource_df['source'],
                                               resource_df['dest'])]
    resource_df = resource_df.drop(['source', 'dest'], axis = 1)


    preferential_df = pd.DataFrame(list(nx.preferential_attachment(G)),
                             columns = ['source', 'dest', 'preferential'])
    preferential_df.index = [(s, d) for s, d in zip(preferential_df['source'],
                                               preferential_df['dest'])]
    preferential_df = preferential_df.drop(['source', 'dest'], axis = 1)

    feature_cols = ['jaccard', 'resource', 'preferential']
    feature_df = pd.concat([jaccard_df,
                            resource_df,
                            preferential_df], axis = 1)

    ### Combine the dfs
    df = future_connections.join(feature_df, how = 'inner')

    ## Split into training and prediction
    train_df = df[~np.isnan(df['Future Connection'])]
    pred_df = df[np.isnan(df['Future Connection'])]

    X = train_df[feature_cols]
    y = train_df['Future Connection']

    log_model = LogisticRegression(random_state=0).fit(X, y)
    train_df['fitted_probs'] = [pred[1] for pred in log_model.predict_proba(X)]

    ## make predictions
    X_pred = pred_df[feature_cols]
    X_pred = X_pred.fillna(0)
    preds = pd.Series([pred[1] for pred in log_model.predict_proba(X_pred)],
                     index = X_pred.index)

    return preds

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.040954
(542, 751)    0.014631
(20, 426)     0.533794
(50, 989)     0.014903
(942, 986)    0.015199
(324, 857)    0.014865
(13, 710)     0.251033
(19, 271)     0.092781
(319, 878)    0.014998
(659, 707)    0.014659
(49, 843)     0.015084
(208, 893)    0.014725
(377, 469)    0.006560
(405, 999)    0.028108
(129, 740)    0.020636
(292, 618)    0.032706
(239, 689)    0.015017
(359, 373)    0.009256
(53, 523)     0.040272
(276, 984)    0.015070
(202, 997)    0.015127
(604, 619)    0.052437
(270, 911)    0.015012
(261, 481)    0.064928
(200, 450)    0.925352
(213, 634)    0.014548
(644, 735)    0.058108
(346, 553)    0.013889
(521, 738)    0.012682
(422, 953)    0.022903
                ...   
(672, 848)    0.015012
(28, 127)     0.957208
(202, 661)    0.013779
(54, 195)     0.999981
(295, 864)    0.014823
(814, 936)    0.014590
(839, 874)    0.015199
(139, 843)    0.014847
(461, 544)    0.011885
(68, 487)     0.011749
(622, 932)    0.014757
(504, 936)    0.017585
(479, 528) 